In [ ]:
---
title: "Lab 2 - Avocado Sales"
author: "Roee Morag"
date: "2024-10-07"
output: 
  html_document:
    theme: "cosmo" 
--- 


In [1]:
import pandas as pd
import numpy as np

In [4]:
avocado_df = pd.read_csv("C:\\Users\\morag\\OneDrive\\Documents\\MSBA\\GSB 544\\Labs\\Labs---GSB-544\\Data\\avocado-updated-2020.csv")

In [11]:
avocado_df.head(n=4)

,date,average_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,geography
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.00,conventional,2015,Albany
1,2015-01-04,1.79,1373.95,57.42,153.88,0.00,1162.65,1162.65,0.00,0.00,organic,2015,Albany
2,2015-01-04,1.00,435021.49,364302.39,23821.16,82.15,46815.79,16707.15,30108.64,0.00,conventional,2015,Atlanta
3,2015-01-04,1.76,3846.69,1500.15,938.35,0.00,1408.19,1071.35,336.84,0.00,organic,2015,Atlanta
4,2015-01-04,1.08,788025.06,53987.31,552906.04,39995.03,141136.68,137146.07,3990.61,0.00,conventional,2015,Baltimore/Washington
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33040,2020-11-29,1.47,1583056.27,67544.48,97996.46,2617.17,1414878.10,906711.52,480191.83,27974.75,organic,2020,Total U.S.
33041,2020-11-29,0.91,5811114.22,1352877.53,589061.83,19741.90,3790665.29,2197611.02,1531530.14,61524.13,conventional,2020,West
33042,2020-11-29,1.48,289961.27,13273.75,19341.09,636.51,256709.92,122606.21,134103.71,0.00,organic,2020,West
33043,2020-11-29,0.67,822818.75,234688.01,80205.15,10543.63,497381.96,285764.11,210808.02,809.83,conventional,2020,West Tex/New Mexico


1. This data set is about avocado prices in different regions/cities of the United States. Each row/observation represents a day of avocado sales in a specific city, i.e Albany, NY or Atlanta, GA. Each region/city has two observations for each date though, one showing their sales of conventional avocados and the other showing their sales of organic avocados. Each of thesscities is also a subset of a region which is a subset of the total United States column. Columns in the data set include average price of the avocados sold, the total volume of avocados sold, total bags, small bags, large bags, and extra large bags sold. The 4046, 4225, and 4770 columns are the price look up codes for smaller, medium, and large avocados and the columns refer to the amount sold each day.

2. As far as cleaning this data set goes, I will clean it as I go through the problems - not quite sure how to organize this until I have intiative with the problems. I will rename some columns, such as 4046, to more intuitive naming schemes.

In [17]:
avocado_df.rename(columns={"4046": "small_medium_avocados", "4225": "large_avocados", "4770": "xlarge_avocados"}, inplace=True)